In [1]:
from pyspark.sql import SparkSession

spark = (SparkSession 
         .builder
         .appName("Lab 1")
         .master("local[5]") #5 threads for parallelism
         .getOrCreate()
        )

ModuleNotFoundError: No module named 'pyspark'

In [12]:
spark._jsc.hadoopConfiguration().set("fs.s3.useRequesterPaysHeader","true")

In [13]:
# Q1

from functools import reduce

errors = [1,10,45,3,5,40]

reduce(lambda s1, s2: s1+s2, map(lambda e: e**2, errors))

3760

In [14]:
# Q2a

from functools import reduce

errors = [1,10,45,3,5,40]

def weighted_mean(obj1, obj2):
  w = obj1["w"] + obj2["w"]
  return ({
      "m" : (obj1["m"] * obj1["w"] + obj2["m"] * obj2["w"]) / w,
      "w" : w
  })
squared_errors = map(lambda e: {"m" : e**2, "w" : 1}, errors)
reduce( weighted_mean, squared_errors )

{'m': 626.6666666666666, 'w': 6}

In [15]:
# Q2b

from functools import reduce

errors = [1,10,45,3,5,40]

def sum_and_n(obj1, obj2) :
  return {
    "sum" : obj1["sum"] + obj2["sum"],
    "n"   : obj1["n"] + obj2["n"]
  }

squared_errors = map(lambda e: {"sum" : e**2, "n" : 1}, errors)
total_sum_and_n = reduce( sum_and_n, squared_errors )
total_sum_and_n["sum"]/total_sum_and_n["n"]

626.6666666666666

In [18]:
# A2

from pyspark.sql.types import FloatType, IntegerType, DateType
from pyspark.sql.functions import substring, col, regexp_replace

# listings_raw = spark.read.csv("s3n://spark-lab-input-data-ensai20202021/airbnb/data-insideairbnb-australia-vic-barwon-south-west-vic-2020-05-26-data-listings.csv.gz", header=True, multiLine=True, escape='"')
# listings_raw = spark.read.csv("s3n://spark-lab-input-data-ensai20202021/airbnb/", header=True, multiLine=True, escape='"')
listings_raw = spark.read.csv("s3a://nrandriamanana/diffusion/formation/data/airbnb/", header=True, multiLine=True, escape='"')

In [19]:
listings_raw.select("price").show()

+-------+
|  price|
+-------+
| $87.00|
| $77.00|
|$200.00|
| $74.00|
|$108.00|
| $95.00|
|$101.00|
|$133.00|
| $97.00|
|$105.00|
| $95.00|
| $75.00|
| $85.00|
|$108.00|
| $84.00|
| $99.00|
| $96.00|
|$106.00|
|$120.00|
|$103.00|
+-------+
only showing top 20 rows



In [64]:
listings = (listings_raw
  .withColumn("beds",     listings_raw["beds"].cast(IntegerType()))
  .withColumn("bedrooms", listings_raw["bedrooms"].cast(IntegerType()))
  .withColumn("time", listings_raw["last_scraped"].cast(DateType()))
  .withColumn("price", regexp_replace('price', '[$\\,]', '').cast(FloatType()))
  .select("id", "beds", "bedrooms", "price", "city", "time")
  .dropna()     
)

In [61]:
listings.groupBy("city").count().sort(col("count").desc()).show()

2022-07-13 14:59:59,521 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz
2022-07-13 14:59:59,561 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz


+--------+-----+
|    city|count|
+--------+-----+
|brussels| 4539|
|    riga| 1776|
|salem-or|  156|
+--------+-----+



2022-07-13 14:59:59,765 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-latvia-riga-riga-2021-12-29-data-listings.csv.gz


In [22]:
listings.show()

+------+----+--------+-----+--------+----------+
|    id|beds|bedrooms|price|    city|      time|
+------+----+--------+-----+--------+----------+
|  2352|   2|       2| 87.0|brussels|2021-12-20|
|  2354|   1|       1| 77.0|brussels|2021-12-20|
| 48180|   2|       2|200.0|brussels|2021-12-20|
| 52796|   1|       1| 74.0|brussels|2021-12-20|
| 96293|   4|       1|133.0|brussels|2021-12-20|
| 96694|   3|       1|105.0|brussels|2021-12-20|
| 97740|   3|       2| 95.0|brussels|2021-12-20|
| 97768|   1|       1| 75.0|brussels|2021-12-20|
|106677|   1|       1| 85.0|brussels|2021-12-20|
|130097|   1|       1| 84.0|brussels|2021-12-20|
|134361|   3|       1| 96.0|brussels|2021-12-20|
|134581|   6|       1|120.0|brussels|2021-12-20|
|148797|   5|       3|100.0|brussels|2021-12-20|
|159836|   3|       1| 49.0|brussels|2021-12-20|
|166510|   1|       1|159.0|brussels|2021-12-20|
|184201|   1|       1| 65.0|brussels|2021-12-20|
|196623|   2|       1|100.0|brussels|2021-12-20|
|198943|   1|       

In [65]:
# Q6

n0 = listings_raw.count()
n  = listings.count()
print("Observations before cleaning:", n0) #
print("Observations  after cleaning:", n, "(change:",  n-n0,")")     # 

2022-07-13 15:01:48,530 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz
2022-07-13 15:01:48,532 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz
2022-07-13 15:01:48,724 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffu

Observations before cleaning: 15084
Observations  after cleaning: 6471 (change: -8613 )


In [56]:
# Q7

print("There are", len(listings_raw.columns), "columns:")
print('\n'.join( ["- "+column for column in listings_raw.columns] ))

There are 75 columns:
- id
- listing_url
- scrape_id
- last_scraped
- name
- description
- neighborhood_overview
- picture_url
- host_id
- host_url
- host_name
- host_since
- host_location
- host_about
- host_response_time
- host_response_rate
- host_acceptance_rate
- host_is_superhost
- host_thumbnail_url
- host_picture_url
- host_neighbourhood
- host_listings_count
- host_total_listings_count
- host_verifications
- host_has_profile_pic
- host_identity_verified
- neighbourhood
- neighbourhood_cleansed
- neighbourhood_group_cleansed
- latitude
- longitude
- property_type
- room_type
- accommodates
- bathrooms
- bathrooms_text
- bedrooms
- beds
- amenities
- price
- minimum_nights
- maximum_nights
- minimum_minimum_nights
- maximum_minimum_nights
- minimum_maximum_nights
- maximum_maximum_nights
- minimum_nights_avg_ntm
- maximum_nights_avg_ntm
- calendar_updated
- has_availability
- availability_30
- availability_60
- availability_90
- availability_365
- calendar_last_scraped
- number_

In [25]:
# Q8

# 3 ways
print(listings_raw.schema)
print(listings_raw.dtypes)
listings_raw.printSchema()

StructType(List(StructField(id,StringType,true),StructField(listing_url,StringType,true),StructField(scrape_id,StringType,true),StructField(last_scraped,StringType,true),StructField(name,StringType,true),StructField(description,StringType,true),StructField(neighborhood_overview,StringType,true),StructField(picture_url,StringType,true),StructField(host_id,StringType,true),StructField(host_url,StringType,true),StructField(host_name,StringType,true),StructField(host_since,StringType,true),StructField(host_location,StringType,true),StructField(host_about,StringType,true),StructField(host_response_time,StringType,true),StructField(host_response_rate,StringType,true),StructField(host_acceptance_rate,StringType,true),StructField(host_is_superhost,StringType,true),StructField(host_thumbnail_url,StringType,true),StructField(host_picture_url,StringType,true),StructField(host_neighbourhood,StringType,true),StructField(host_listings_count,StringType,true),StructField(host_total_listings_count,Stri

In [26]:
# Q9

In [32]:
from timeit import default_timer as t

## version 1 ------------------

start = t()

listings.summary("count", "min", "max").show()

print ("Execution time for sumary(\"count\"...):", t()-start, "s")

## version 2 ------------------

start = t()

listings.summary().show()

print ("Execution time for sumary():", t()-start, "s")


2022-07-13 14:50:52,438 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz
2022-07-13 14:50:52,554 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz
2022-07-13 14:50:52,693 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffu

+-------+--------+----+--------+------+--------+
|summary|      id|beds|bedrooms| price|    city|
+-------+--------+----+--------+------+--------+
|  count|    6471|6471|    6471|  6471|    6471|
|    min|10008009|   1|       1|   8.0|brussels|
|    max| 9981899|  50|      40|5000.0|salem-or|
+-------+--------+----+--------+------+--------+

Execution time for sumary("count"...): 0.6172656689996074 s


2022-07-13 14:50:53,079 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz
2022-07-13 14:50:53,220 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz
2022-07-13 14:50:53,419 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffu

+-------+--------------------+------------------+------------------+------------------+--------+
|summary|                  id|              beds|          bedrooms|             price|    city|
+-------+--------------------+------------------+------------------+------------------+--------+
|  count|                6471|              6471|              6471|              6471|    6471|
|   mean| 3.192046149126874E7|2.0100448153299335|1.4546437953948386| 96.29377221449545|    null|
| stddev|1.5987299928039854E7|1.6912515639919852|1.0939229318497001|223.17858006434983|    null|
|    min|            10008009|                 1|                 1|               8.0|brussels|
|    25%|         1.9120099E7|                 1|                 1|              45.0|    null|
|    50%|         3.4994211E7|                 2|                 1|              66.0|    null|
|    75%|         4.5584955E7|                 2|                 2|              98.0|    null|
|    max|             9981899|

In [28]:
# Q10

In [29]:
## version 1 ------------------

from timeit import default_timer as t
from pyspark.sql.functions import mean

start = t()

listings.filter(col("city")=="Fairhaven").select(mean('beds')).show()

print ("Execution time:", t()-start, "s")

2022-07-13 14:50:26,757 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz
2022-07-13 14:50:26,758 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz
2022-07-13 14:50:26,905 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffu

+---------+
|avg(beds)|
+---------+
|     null|
+---------+

Execution time: 0.6358329180002329 s


In [33]:
## version 2 ------------------

from timeit import default_timer as t

start = t()

listings.filter(col("city")=="brussels").select("beds").summary("mean").show()

print ("Execution time:", t()-start, "s")

2022-07-13 14:51:57,176 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz
2022-07-13 14:51:57,230 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz


+-------+-----------------+
|summary|             beds|
+-------+-----------------+
|   mean|1.834324741132408|
+-------+-----------------+

Execution time: 0.6423636129984516 s


2022-07-13 14:51:57,381 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-latvia-riga-riga-2021-12-29-data-listings.csv.gz


In [38]:
## version 3 ------------------

from timeit import default_timer as t
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.stat    import Summarizer

start = t()

vectorizer = VectorAssembler(
    inputCols     = ["beds"],
    outputCol     = "features"
)
summarizer = Summarizer.metrics("mean")
listings_vec = vectorizer.transform(listings.filter(col("city")=="brussels"))
listings_vec.select( summarizer.summary( listings_vec.features) ).show(truncate=False)


print ("Execution time:", t()-start, "s")

2022-07-13 14:53:19,766 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz
2022-07-13 14:53:19,768 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz
2022-07-13 14:53:19,890 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffu

+--------------------------------+
|aggregate_metrics(features, 1.0)|
+--------------------------------+
|{[1.834324741132409]}           |
+--------------------------------+

Execution time: 0.6643451919990184 s


In [39]:
## version 4 ---------------
sc.install_pypi_package("pandas==0.25.1")

NameError: name 'sc' is not defined

In [40]:
from timeit import default_timer as t

start = t()
listings.filter(col("city")=="brussels").select('beds').toPandas().mean()

print ("Execution time:", t()-start, "s")

2022-07-13 14:53:29,810 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz
2022-07-13 14:53:29,854 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz


Execution time: 0.642377268999553 s


2022-07-13 14:53:30,096 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-latvia-riga-riga-2021-12-29-data-listings.csv.gz


In [41]:
# Q11

from pyspark.sql.functions import mean
price_mean = listings.select(mean("price"))        # this is a (remote) data frame
price_mean = price_mean.first()                    # this is a (local) row
price_mean = price_mean[0]                         # this is a (local) number

2022-07-13 14:53:38,487 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz
2022-07-13 14:53:38,594 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz
2022-07-13 14:53:39,152 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffu

In [42]:
price_mean

96.29377221449545

In [43]:
from pyspark.sql.functions import sum
tss = listings.withColumn("squared_error", (col("price")-price_mean)**2).select(sum("squared_error"))
tss = tss.first()
tss = tss[0]

2022-07-13 14:53:44,992 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz
2022-07-13 14:53:45,299 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-latvia-riga-riga-2021-12-29-data-listings.csv.gz
2022-07-13 14:53:45,368 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/form

In [44]:
tss

322262150.5390205

In [45]:
# Q12

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

vectorizer = VectorAssembler( # copy-pasted from previous section...
    inputCols     = ["beds", "bedrooms"], # ... but without price
    outputCol     = "features",
    handleInvalid = 'skip'
)

listings_vec = vectorizer.transform(listings)

regressor = LinearRegression(featuresCol="features", labelCol="price")
model     = regressor.fit(listings_vec)

2022-07-13 14:53:45,850 WARN util.Instrumentation: [7168c5ff] regParam is zero, which might cause numerical instability and overfitting.
2022-07-13 14:53:46,023 WARN netlib.InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
2022-07-13 14:53:46,025 WARN netlib.InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
2022-07-13 14:53:46,100 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz
2022-07-13 14:53:46,104 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but f

In [46]:
model.coefficients

DenseVector([12.6064, 13.8308])

In [47]:
model.intercept

50.835461713709314

In [48]:
listings_vec.show()

+------+----+--------+-----+--------+----------+---------+
|    id|beds|bedrooms|price|    city|      time| features|
+------+----+--------+-----+--------+----------+---------+
|  2352|   2|       2| 87.0|brussels|2021-12-20|[2.0,2.0]|
|  2354|   1|       1| 77.0|brussels|2021-12-20|[1.0,1.0]|
| 48180|   2|       2|200.0|brussels|2021-12-20|[2.0,2.0]|
| 52796|   1|       1| 74.0|brussels|2021-12-20|[1.0,1.0]|
| 96293|   4|       1|133.0|brussels|2021-12-20|[4.0,1.0]|
| 96694|   3|       1|105.0|brussels|2021-12-20|[3.0,1.0]|
| 97740|   3|       2| 95.0|brussels|2021-12-20|[3.0,2.0]|
| 97768|   1|       1| 75.0|brussels|2021-12-20|[1.0,1.0]|
|106677|   1|       1| 85.0|brussels|2021-12-20|[1.0,1.0]|
|130097|   1|       1| 84.0|brussels|2021-12-20|[1.0,1.0]|
|134361|   3|       1| 96.0|brussels|2021-12-20|[3.0,1.0]|
|134581|   6|       1|120.0|brussels|2021-12-20|[6.0,1.0]|
|148797|   5|       3|100.0|brussels|2021-12-20|[5.0,3.0]|
|159836|   3|       1| 49.0|brussels|2021-12-20|[3.0,1.0

In [49]:
# Q13

dir(model)
dir(model.summary)

['__class__',
 '__del__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_call_java',
 '_create_from_java_class',
 '_java_obj',
 '_new_java_array',
 '_new_java_obj',
 'coefficientStandardErrors',
 'degreesOfFreedom',
 'devianceResiduals',
 'explainedVariance',
 'featuresCol',
 'labelCol',
 'meanAbsoluteError',
 'meanSquaredError',
 'numInstances',
 'objectiveHistory',
 'pValues',
 'predictionCol',
 'predictions',
 'r2',
 'r2adj',
 'residuals',
 'rootMeanSquaredError',
 'tValues',
 'totalIterations']

In [50]:
model.summary.r2

0.02264106645437436

In [51]:
# model.summary.explainedVariance
# je voulais recalculer le R2, mais je passe

In [73]:
# Q14

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml import Pipeline
from pyspark.sql.functions import year

vectorizer = VectorAssembler(
    inputCols     = ["beds", "bedrooms"],
    outputCol     = "features",
    handleInvalid = 'skip'
)
regressor = LinearRegression(featuresCol="features", labelCol="price")
pipeline  = Pipeline(stages = [vectorizer, regressor])

model_2021 = pipeline.fit(listings.filter(year(listings.time)==2021))
#model_2020 = pipeline.fit(listings.filter(year(listings.time)==2020))

2022-07-13 15:03:49,969 WARN util.Instrumentation: [d847ddd3] regParam is zero, which might cause numerical instability and overfitting.
2022-07-13 15:03:50,102 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz
2022-07-13 15:03:50,133 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz
2022-07-13 15:03:50,323 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedro

In [66]:
listings.groupBy("time").count().show()

2022-07-13 15:03:09,490 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz
2022-07-13 15:03:09,573 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz


+----------+-----+
|      time|count|
+----------+-----+
|2021-12-20| 4695|
|2021-12-29| 1776|
+----------+-----+



2022-07-13 15:03:09,697 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-latvia-riga-riga-2021-12-29-data-listings.csv.gz


In [67]:
model_2020.stages[1].coefficients

NameError: name 'model_2020' is not defined

In [71]:
model_2021.stages[1].coefficients

DenseVector([12.6064, 13.8308])

In [68]:
listings.filter(year(listings.time)==2021).show()

+------+----+--------+-----+--------+----------+
|    id|beds|bedrooms|price|    city|      time|
+------+----+--------+-----+--------+----------+
|  2352|   2|       2| 87.0|brussels|2021-12-20|
|  2354|   1|       1| 77.0|brussels|2021-12-20|
| 48180|   2|       2|200.0|brussels|2021-12-20|
| 52796|   1|       1| 74.0|brussels|2021-12-20|
| 96293|   4|       1|133.0|brussels|2021-12-20|
| 96694|   3|       1|105.0|brussels|2021-12-20|
| 97740|   3|       2| 95.0|brussels|2021-12-20|
| 97768|   1|       1| 75.0|brussels|2021-12-20|
|106677|   1|       1| 85.0|brussels|2021-12-20|
|130097|   1|       1| 84.0|brussels|2021-12-20|
|134361|   3|       1| 96.0|brussels|2021-12-20|
|134581|   6|       1|120.0|brussels|2021-12-20|
|148797|   5|       3|100.0|brussels|2021-12-20|
|159836|   3|       1| 49.0|brussels|2021-12-20|
|166510|   1|       1|159.0|brussels|2021-12-20|
|184201|   1|       1| 65.0|brussels|2021-12-20|
|196623|   2|       1|100.0|brussels|2021-12-20|
|198943|   1|       

In [69]:
model_2021.stages[1].intercept

50.835461713709314

In [72]:
model_2020.stages[1].intercept

NameError: name 'model_2020' is not defined

In [53]:
listings.groupBy(year("time")).agg(mean("price")).show()

2022-07-13 14:53:50,977 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-united-states-or-salem-or-2021-12-20-data-listings.csv.gz
2022-07-13 14:53:51,001 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffusion/formation/data/airbnb/temp_data-insideairbnb-belgium-bru-brussels-2021-12-20-data-listings.csv.gz
2022-07-13 14:53:51,154 WARN csv.CSVHeaderChecker: CSV header does not conform to the schema.
 Header: id, last_scraped, bedrooms, beds, price, 
 Schema: id, last_scraped, bedrooms, beds, price, city
Expected: city but found: 
CSV file: s3a://nrandriamanana/diffu

+----------+-----------------+
|year(time)|       avg(price)|
+----------+-----------------+
|      2021|96.29377221449545|
+----------+-----------------+

